In [1]:
# Necessary imports
import pandas as pd
import re
import numpy as np
import glob
import os

# path = "nationa"
path_raw = "national_raw/"
# path_processed = "Data/processed/"

files = glob.glob(path_raw + "*.csv")
#specify path to files.
files
len(files)

310

## Conversion functions

In [2]:
def most_recent_data_format(temp, file, indicator, indicator_category, unit_of_measure):
    """One of the formats in the CSV's is the 'most recent data format' with just one 
    value (the most recent one known by UNAIDS), this function checks the number of columns.
    Selects the columns to keep first 3. Then locs that dataframe, re-names a columns in place. 
    Finally calls the melt function to shape the file into the right format. It returns
    the correctly shaped file to the loop which concats it to the rest.
    
    -- Input: 
    temp = a UNAIDS CSV file with the most recent format read in the for loop that loops throught the files. 
    file = the name of the file to check for bugs if the assert is trigged
    indicator = the name of the indicator
    indicator_category = the name of the key population 
    unit_of_measure = can be either a number or a percentage
    
    -- Output: 
    Returns a correctly formatted dataframe which can be concatted to the final dataframe. 
    """
    
    try:
        assert len(temp.columns) == 3 #Make sure there are just 3 columns so we don't lose any important information.
    except:
        print('There are more than 3 columns in iteration {} please check it out'.format(file))
        print(temp.columns)
        
    columns_to_keep = temp.columns[0:2]
    temp = temp.loc[:, tuple(columns_to_keep)] #ignore the irrelevant columns
    temp.rename(columns={'Most recent data as of 2017': '2017'}, inplace=True)
    
    temp_final = melt(temp, indicator, indicator_category, unit_of_measure)
    
    return temp_final

def multiple_years(temp, file, indicator, indicator_category, unit_of_measure):
    """The other format provided by UNAIDS is data from multiple years. Every year 
    has 3 values (upper, lower and realistic). Which yield years*3 columns + 1 for the country.
    Then the columns per category are initiallized for get the right subset of the df at a later stage. 
    The for loop appends the correct columns to the columns_lists. 
    3 temp df are created by locing the right columns. Next the right keys (columns) are matched per dataset. 
    These were for instance 2012_lower and should be 2012, replacement is done in place. 
    Next all dataframes are melted into the right format and the estimate is added to the indicator category. 
    All files are concatted and finnaly returned to the for loop that loops over the folder. 
    
    -- Input: 
    temp = a UNAIDS CSV file with the multiple years format read in the for loop that loops throught the files. 
    file = the name of the file to check for bugs if the assert is trigged
    indicator = the name of the indicator
    indicator_category = the name of the key population 
    unit_of_measure = can be either a number or a percentage
    
    -- Output: 
    Returns a correctly formatted dataframe which can be concatted to the final dataframe. 
    """
    temp.rename(columns=lambda x: x.strip(), inplace=True)
    
    columns_lower = ['Country']
    columns_upper = ['Country']
    columns_realistic = ['Country']
    #print(columns_lower)
    for i in range(1, len(temp.columns)):
        if 'upper' in list(temp.columns)[i]:
            #print('upper', i)
            columns_upper.append(list(temp.columns)[i])
        elif 'lower' in list(temp.columns)[i]:
            columns_lower.append(list(temp.columns)[i])
            #print('lower', i)
        elif 'upper' or 'lower' not in list(temp.columns)[i]:
            columns_realistic.append(list(temp.columns)[i])
            #print('years', i)

    temp_realistic = temp.loc[:, tuple(columns_realistic)]
    temp_lower = temp.loc[:, tuple(columns_lower)]
    temp_upper = temp.loc[:, tuple(columns_upper)]

    keys_lower = columns_lower
    values_realistic = columns_realistic
    dict_lower = dict(zip(keys_lower, values_realistic))
    temp_lower.rename(columns=dict_lower, inplace=True)
    
    keys_upper = columns_upper
    values_realistic = columns_realistic
    dict_upper = dict(zip(keys_upper, values_realistic))
    temp_upper.rename(columns=dict_upper, inplace=True)
    
#     temp_realistic = melt(temp_realistic, indicator, str(indicator_category)+' realistic estimate', unit_of_measure)
    temp_realistic = melt(temp_realistic, indicator, str(indicator_category), unit_of_measure)
    temp_lower = melt(temp_lower, indicator, str(indicator_category)+' lower bound', unit_of_measure)
    temp_upper = melt(temp_upper, indicator, str(indicator_category)+' upper bound', unit_of_measure)
    #print(temp_realistic.columns)
    #print(temp_upper.columns)
    #print(temp_lower.columns)
    
#     temp_final = pd.concat([temp_realistic, temp_lower, temp_upper])
    temp_final = pd.concat([temp_realistic])
                      
    #return temp_realistic, temp_lower, temp_upper
    return temp_final

def melt(temp, indicator, indicator_category, unit_of_measure):
    """
     -- Input: 
    temp = a UNAIDS CSV file with the multiple years format read in the for loop that loops throught the files. 
    file = the name of the file to check for bugs if the assert is trigged
    indicator = the name of the indicator
    indicator_category = the name of the key population 
    unit_of_measure = can be either a number or a percentage
    
    A function to melt all given dataframes in the right format.
    -- Output:
    Melted dataframe in the right format. 
         """
    #id_vars: define identifier variables
    identifiers = "Country"
    #var_name: scalar; now distributed in multiple columns, will be one column
    scalar = 'date_value'
    #value_name: the values that are in the columns of the scalar get their own column
    valuename = 'measure_value'
    
    temp = pd.melt(temp, id_vars = identifiers, var_name=scalar, value_name = valuename)
    temp['indicator_category'] = indicator_category
    temp['indicator'] = indicator
    temp['unit_of_measure'] = unit_of_measure
    return temp

## Loop over all files and execute steps

In [3]:
"""The cell to get all files in the right format. It loops over a specified files folder.
A some variables are deducted from the file_name, then they are shaped into the right format
by the specified helper functions. Finally all files are concated into a final total dataframe.
"""

df = pd.DataFrame(columns = ['Country', 'date_value', 'measure_value', 
                             'indicator_category', 'indicator', 'unit_of_measure'])


for file in range(0, len(files)):
    try:
        file_name = files[file].split('\\')[-1]
        assert len(file_name.split('_')) == 3 #Make sure all files are seperated by 3 _'s. 
        #print(file_name,' is not seperated correctly')
    except:
        print(file_name,' is not seperated correctly, please check it out')

    indicator_category = file_name.split("_")[2]
    indicator_category = indicator_category.split('.')[0]

    indicator = file_name.split("_")[0]

    unit_of_measure = file_name.split("_")[1]

    #Import the file
    temp = pd.read_csv(files[file])

    # Select the right format, first is 'Most recent format'
    # Select the right year if another (i.e. 2018) is applicable. 
    if "Most recent data as of 2017" in list(temp.columns):
        temp = most_recent_data_format(temp, file, indicator, indicator_category, unit_of_measure)
    #Select the multiple year format.
    else:
        try:
            temp = multiple_years(temp, file, indicator, indicator_category, unit_of_measure)
        except:
            print(file_name)
    df = pd.concat([df, temp])
#     print(file_name, file)

## Clean the file for syntax mistakes

In [4]:
df['indicator'] = df['indicator'].str.replace('  ',' ').str.lstrip().str.rstrip()

"""Regex to keep only integers and floats"""
non_decimal = re.compile(r'[^\d.]+')
df['measure_value'] = df['measure_value'].apply(lambda x: non_decimal.sub('',x) if type(x) == str else x)

"""Convert all to numeric values"""
df['measure_value'] = pd.to_numeric(df['measure_value'],errors='coerce')

"""Drop all rows where measure value contains nans"""
df = df[np.isfinite(df['measure_value'])]

"""Double check if measure value only contains numeric values. Errors are printed"""
for number in df['measure_value']:
    try:
        int(number)
    except Exception:
        print(number)       
        
        
#remove whitespaces
df['indicator_category'] = df['indicator_category'].str.replace('  ',' ').str.lstrip().str.rstrip()

df.loc[:, 'indicator'] = df.indicator.str.replace('Prevalence', 'prevalence')

## Clean the fil for typo's and add codes

In [5]:
"""Read csv with countries and isocodes"""
# general_dir = "/home/science/Projects/aidsfonds/3_unaidsupdate_Mei_2018/Data/"
countries = pd.read_csv('region_codes.csv',encoding = "ISO-8859-1", delimiter = ';')

"""Create dictionary with country as key and ISO as value"""
isocodes = countries[['Name','ISO3']].set_index('Name')
isocodes = isocodes.to_dict(orient='dict')
isocodes = isocodes['ISO3'].items()
isocodes = dict(isocodes)

"""Create dictionary with to replace the wrong matches"""
nadict = {}
# nadict['Czech Republic'] = 'Czechia'
nadict["Democratic People\'s Republic of Korea"] = "North korea"
nadict["Democratic Republic of the Congo"] = "Republic of congo"
# nadict["Korea (Republic of)"] = "South korea"
# nadict["Korea (Democratic People's Republic of)"] = "North korea"
nadict["Congo"] = "Republic of congo"
nadict["Republic of Korea"] = "South korea"
nadict['Republic of Moldova'] = "Moldova"
nadict['Swaziland'] = "Eswatini"
nadict['The former Yugoslav Republic of Macedonia'] = "Macedonia"
nadict['United Kingdom'] = "United Kingdom of Great Britain and Northern Ireland"
nadict['United Republic of Tanzania'] = "Tanzania"
nadict['United States'] = "United States of America"
nadict['United States Virgin Islands'] = "Virgin Islands (U.S.)"
nadict['Global'] = 'Global'
nadict['Viet Nam'] = 'Vietnam'
nadict["Côte d'Ivoire"] = 'Ivory coast'
nadict["North Macedonia"] = 'Macedonia'
nadict["Czechia"] = 'Czech Republic'
nadict["Tanzania, United Republic of"] = 'United Republic of Tanzania'


"""Replace wrong country name with right country name"""
df = df.replace(nadict)

"""Delete Global country name, can not be mapped in Zoom"""
df = df[df['Country'] != 'Global']

"""Create new column with ISO3-codes"""
df['ISO'] = df['Country'].map(isocodes)


# df['Country'] = df['Country'].replace("Tanzania, United Republic of", "United Republic of Tanzania")

"""Create new column with ISO3-codes"""
df['ISO'] = df['Country'].str.lower().map(isocodes)

## Filter relevant data

In [6]:
#get rid of all values that are not just years
years = ['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
 '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
 '2016', '2017', '2018']

df = df[df['date_value'].isin(years)].reset_index()
del df['index']

In [7]:
df.columns = [['Country', 'Date', 'measure_value', 'legend', 'Indicator', 'Value_format', 'ISO']]

## Store data

In [8]:
df.to_csv('unaids_national_25-08-2020.csv', sep=',', encoding = 'UTF-8', index = False)